# 4_model_analysis.ipynb

- **Objectif** :
Analyser les modèles enregistrés dans MLflow,
comparer leurs performances et visualiser les métriques clés.
---


In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from joblib import load


# 1️⃣ Récupération des runs MLflow


In [ ]:
mlflow.set_tracking_uri("file:./mlruns")
experiment_name = "Churn_Prediction_Models"
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

In [ ]:
if experiment is None:
    raise ValueError(f"⚠️ Aucun experiment MLflow trouvé avec le nom : {experiment_name}")

In [ ]:
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
print(f"✅ {len(runs)} runs trouvés pour l'expérience '{experiment_name}'")

# 2️⃣ Synthèse des performances

In [ ]:
# ==========================================================
# 2️⃣ Synthèse des performances
# ==========================================================
metrics_summary = runs[["tags.model_name", "tags.imbalance_strategy",
                        "metrics.f1_score", "metrics.accuracy", "run_id"]].sort_values("metrics.f1_score", ascending=False)
metrics_summary.columns = ["Model", "Strategy", "F1_score", "Accuracy", "Run_ID"]

In [ ]:
display(metrics_summary.head(10))

# 3️⃣ Visualisation comparée (F1 & Accuracy)

In [ ]:
# ==========================================================
# 3️⃣ Visualisation comparée (F1 & Accuracy)
# ==========================================================
plt.figure(figsize=(8,5))
sns.barplot(data=metrics_summary, x="Model", y="F1_score", hue="Strategy")
plt.title("📊 Comparaison du F1-score par modèle et stratégie")
plt.ylabel("F1-score")
plt.legend(title="Stratégie")
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=metrics_summary, x="Model", y="Accuracy", hue="Strategy")
plt.title("📈 Comparaison de l’Accuracy par modèle et stratégie")
plt.ylabel("Accuracy")
plt.legend(title="Stratégie")
plt.show()

# 4️⃣ Sélection du meilleur modèle

In [ ]:
# ==========================================================
# 4️⃣ Sélection du meilleur modèle
# ==========================================================
best_run = metrics_summary.iloc[0]
print(f"🏆 Meilleur modèle : {best_run['Model']} ({best_run['Strategy']})")
print(f"F1 = {best_run['F1_score']:.3f} | Accuracy = {best_run['Accuracy']:.3f}")

## Charger le modèle depuis MLflow

In [ ]:
# Charger le modèle depuis MLflow
model_uri = f"runs:/{best_run['Run_ID']}/model"
best_model = mlflow.sklearn.load_model(model_uri)

## Charger les données test

In [ ]:
# Charger les données test
X_test = load("data/X_test.pkl")
y_test = load("data/y_test.pkl")

# 5️⃣ Visualisations de performance du meilleur modèle

In [ ]:
# ==========================================================
# 5️⃣ Visualisations de performance du meilleur modèle
# ==========================================================
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:,1]

## Matrice de confusion

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm).plot(cmap="Blues")
plt.title(f"Confusion Matrix - {best_run['Model']} ({best_run['Strategy']})")
plt.show()

## Courbe ROC

In [ ]:
# Courbe ROC
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0,1], [0,1], linestyle="--", color="gray")
plt.title(f"ROC Curve - {best_run['Model']} ({best_run['Strategy']})")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

# 6️⃣ Export de la synthèse

In [ ]:
# ==========================================================
# 6️⃣ Export de la synthèse
# ==========================================================
metrics_summary.to_csv("reports/model_comparison_results.csv", index=False)
print("💾 Résultats sauvegardés dans reports/model_comparison_results.csv")

# 7️⃣ Rapport rapide

In [ ]:
# ==========================================================
# 7️⃣ Rapport rapide
# ==========================================================
import markdown
summary_text = f"""
# 🧾 Rapport de comparaison des modèles

**Expérience MLflow :** {experiment_name}

**Nombre de runs :** {len(runs)}

**Meilleur modèle :** {best_run['Model']} ({best_run['Strategy']})
- F1-score : {best_run['F1_score']:.3f}
- Accuracy : {best_run['Accuracy']:.3f}

Les figures ci-dessus montrent la comparaison entre les trois modèles et les différentes stratégies d'équilibrage.
Le meilleur compromis entre précision et rappel est obtenu avec **{best_run['Model']}** en mode **{best_run['Strategy']}**.
"""

In [ ]:
with open("reports/model_comparison.md", "w", encoding="utf-8") as f:
    f.write(summary_text)

In [ ]:
print("📘 Rapport enregistré : reports/model_comparison.md")